# Load the database and model

In [1]:
include("src/GTAPinJulia.jl")
using .GTAPinJulia

#   Choose a database and model options

if true
    inFolder    = "Data/DBUG0"
    BaseName    = "DBUG0"
    resName     = :USA
    capName     = :cap
    nrsFact     = [:nrs]
    savfClosure = :capFix
else
    inFolder    = "Data/3x3"
    BaseName    = "3x3"
    resName     = :USA
    capName     = :CAP
    nrsFact     = [:xxx]
    savfClosure = :capFix
end
inscale  = 1e-6
popscale = 1e-3

#   Get the GTAP data, initialize the model variables and calibrate the model parameters
data = GTAPinJulia.load_gtap_data(; inFolder = inFolder, BaseName = BaseName, inscale = inscale, popscale = popscale, resName = resName, capName = capName, nrsFact = nrsFact)

#   Define the model
#   The capital closure options are:
#    1. RoRFlex--standard GTAP closure with foreign savings to equate expected rates of return
#    2. capFix--Foreign savings fixed (wrt to model numéraire)
#    3. capShrFix--Foreign savings fixed wrt to GDP (requires a residual region)

GTAPModel = GTAPinJulia.GTAPModel(data, resName = resName, capName = capName, savfClosure = savfClosure)


A JuMP Model
├ solver: Path 5.0.03
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 56867
├ num_constraints: 44676
│ ├ Vector{JuMP.NonlinearExpr} in MOI.Complements: 10847
│ ├ Vector{JuMP.AffExpr} in MOI.Complements: 2191
│ ├ Vector{JuMP.QuadExpr} in MOI.Complements: 9266
│ └ JuMP.VariableRef in MOI.EqualTo{Float64}: 22372
└ Names registered in the model
  └ :Walras, :chi_qe, :chisave, :conshr, :eq_Walras, :eq_chisave, :eq_conshr, :eq_fincome, :eq_gdpfc, :eq_indtax, :eq_kb, :eq_ke, :eq_pca, :eq_pcif, :eq_pds, :eq_pe, :eq_peb, :eq_pes, :eq_pfa, :eq_pfact, :eq_pfactw, :eq_pfd, :eq_pfe, :eq_pfm, :eq_pfob, :eq_pga, :eq_pgd, :eq_pgm, :eq_pgov, :eq_pia, :eq_pid, :eq_pim, :eq_pint, :eq_pinv, :eq_pmds, :eq_pms, :eq_po, :eq_ppa, :eq_ppd, :eq_ppm, :eq_ps, :eq_psave, :eq_pt, :eq_ptrans, :eq_pva, :eq_qc, :eq_qca, :eq_qds, :eq_qe, :eq_qfa, :eq_qfd, :eq_qfe, :eq_qfm, :eq_qga, :eq_qgd, :eq_qgm, :eq_qia, :eq_qid, :eq_qim, :eq_qint, :eq_qinv, :eq_qms, :eq_qo, :eq_qpa, :eq_qpd, :eq_qpm, :eq_qsave, 

#   Fix the standard exogenous variables:
- tax rates
- trade margins
- factor supply shifter
- population

In [2]:
#   Fix the exogenous variables
GTAPinJulia.fix.(GTAPModel[:ptax], data[:ptax0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tfe], data[:tfe0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tinc],data[:tinc0]; force=true)

GTAPinJulia.fix.(GTAPModel[:tfd], data[:tfd0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tfm], data[:tfm0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tpd], data[:tpd0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tpm], data[:tpm0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tgd], data[:tgd0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tgm], data[:tgm0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tid], data[:tid0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tim], data[:tim0]; force=true)

GTAPinJulia.fix.(GTAPModel[:txs], data[:txs0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tmarg], data[:tmarg0]; force=true)
GTAPinJulia.fix.(GTAPModel[:tms], data[:tms0]; force=true)

GTAPinJulia.fix.(GTAPModel[:chi_qe], 1.0; force=true)
GTAPinJulia.fix.(GTAPModel[:pop], data[:pop0]; force=true)
GTAPinJulia.fix.(GTAPModel[:pnum], 1.0; force=true)


#   Run the benchmark simulation

In [5]:
# Benchmark
GTAPinJulia.fix(GTAPModel[:pnum], 1; force=true)

GTAPinJulia.set_attribute(GTAPModel, "cumulative_iteration_limit", 10_000)
GTAPinJulia.optimize!(GTAPModel)

#   Include the output functions
# include("src/reporting.jl")

Reading options file C:\Users\vandermd\AppData\Local\Temp\jl_B364.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 22304

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             5.5489e-06             0.0e+00 (eq_yg[SAS)
    1     1     0 22304 2.0196e-07  1.0e+00    0.0e+00 (eq_qfe[nsk,WND-a,RUS)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 2.0196e-07           I 0.0e+00 1.4e-07 (eq_qfe[nsk,WND-)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 2
Gradient Evaluations. . 2
Basis Time. . . . . . . 0.031000
Total Time. . . . . . . 0.109000
Residual. . . . . . . . 2.019579e-07
Posts

LoadError: LoadError: ParseError:
# Error @ v:\Julia\GTAPinJulia\src\reporting.jl:48:219
    for a∈acts
        values = @. outscale*value(GTAPMod[:qfd][comm,a,reg]*GTAPMod[:pds][comm,reg]*data[:qfd0][comm,a,reg]*data[:pds0][comm,reg]+GTAPMod[:qfm][comm,a,reg]*GTAPMod[:pms][comm,reg]*data[:qfm0[comm,a,reg]*pms0[comm,reg])
#                                                                                                                                                                                                                         └ ── Expected `]`
in expression starting at v:\Julia\GTAPinJulia\src\reporting.jl:48

#   Run the homogeneity test

In [4]:
GTAPinJulia.set_attribute(GTAPModel, "cumulative_iteration_limit", 10_000)

GTAPinJulia.fix(GTAPModel[:pnum], 1.2; force=true)

GTAPinJulia.optimize!(GTAPModel)

Reading options file C:\Users\vandermd\AppData\Local\Temp\jl_2D5A.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 22304

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.5129e-01             0.0e+00 (eq_Walra)
    1     1     0 22304 2.9722e-03  1.0e+00    0.0e+00 (eq_ror)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 2.9722e-03           I 0.0e+00 3.0e-03 (eq_ror)
    1     1     3     3 1.5445e-09  1.0e+00 SO 0.0e+00 1.4e-09 (eq_qfe[nsk,WND-)

Major Iterations. . . . 1
Minor Iterations. . . . 1
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 3
Gradient Evaluations. . 3
Basis Time. . . . . . . 0.062000
Total Time. . . . 